# Negative Prompting

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [3]:
async def get_response(prompt):
    """Helper function to get response from the language model."""
    return await llm.ainvoke(prompt)

#Basic

In [4]:
import nest_asyncio
nest_asyncio.apply()
import asyncio

In [7]:
from langchain_core.prompts import PromptTemplate

neg_prompt_template = PromptTemplate(
    input_variables=["topic"],
    template="""
    Provide a brief explanation of the topic "{topic}".
    Do Not include any of the following in your response:
    1) Technical jargon or complex terminology
    2) Historical background or dates
    3) Comparisons with other topics
    Your explanation should be concise, clear, and focused solely on the topic at hand.
    """
)
response = asyncio.run(get_response(neg_prompt_template.format(topic="photosynthesis")))
print(response.content)

Here is a brief explanation of photosynthesis:

Photosynthesis is the process by which plants, algae, and some bacteria make their own food from sunlight. They use energy from the sun, water, and a gas called carbon dioxide to produce a type of sugar that gives them energy. This process also releases oxygen into the air, which is important for animals like us to breathe. In simple terms, photosynthesis is how plants grow and thrive by turning sunlight into a source of energy.


#Specifying Exclusions

In [8]:
exclusion_prompts = PromptTemplate(
    input_variables=["topic", "exclude"],
    template="""
    Write a short paragraph about "{topic}".
    Important: DO NOT mention or reference anything related to "{exclude}".
    """
)
response = asyncio.run(get_response(
    exclusion_prompts.format(
        topic="the benefits of exercise",
        exclude="weight loss or body image"
    )
))
print(response.content)

Regular exercise has numerous benefits for both physical and mental well-being. Exercise has been shown to improve overall health by strengthening the heart and lungs, reducing the risk of chronic diseases, and increasing flexibility and mobility. Additionally, exercise has been linked to improved mental health, reducing symptoms of anxiety and depression, and improving sleep quality. Exercise has also been shown to boost cognitive function, improving concentration and memory, and increasing overall sense of energy and vitality. Furthermore, exercise can provide a sense of accomplishment and improved self-esteem, as individuals set and achieve goals and push themselves to new heights.


#Implementing Constraints

In [9]:
constraint_prompt = PromptTemplate(
    input_variables=["topic","style", "exclude_words"],
    template="""
    Write a {style} description of topic "{topic}".
    Constraints:
    1. DO NOT include any of the following words: {exclude_words}.
    2. Keep the description under 100 words.
    3. DO NOT use analogies or metaphores.
    4. Focus only on factual information.
    """
)
response = asyncio.run(get_response(
    constraint_prompt.format(
        topic="artificial intelligence",
        style="technical",
        exclude_words="robot, human-like, science fiction"
    )
))
print(response.content)

Artificial intelligence (AI) refers to the development of computer systems that can process and analyze data, make decisions, and take actions without being explicitly programmed. AI systems utilize complex algorithms and statistical models to learn from data, identify patterns, and recognize relationships. These systems can perform tasks such as image recognition, natural language processing, and predictive analytics. AI can be applied in various domains, including healthcare, finance, and transportation, to improve efficiency, accuracy, and decision-making. AI systems can be divided into categories, including machine learning, deep learning, and decision support systems.


#Evaluation and refinement

In [14]:
import re

def evaluate_output(output, constraints):
    """Evaluate if the output meets the given constraints."""
    results = {}
    for constraint, check_func in constraints.items():
        results[constraint] = check_func(output)
    return results

# Define some example constraints
constraints = {
    "word_count": lambda x: len(x.split()) <= 100,
    "no_excluded_words": lambda x: all(word not in x.lower() for word in ["robot", "human-like", "science fiction"]),
    "no_analogies": lambda x: not re.search(r"\b(as|like)\b", x, re.IGNORECASE) 

}

# Evaluate the previous output
evaluation_results = evaluate_output(response, constraints)
print("Evaluation results:", evaluation_results)

AttributeError: 'AIMessage' object has no attribute 'split'

In [ ]:
if not all(evaluation_results.values()):
    refined_prompt = constraint_prompt.format(
        topic="artificial intelligence",
        style="technical and concise",  # Added 'concise' to address word count
        excluded_words="robot, human-like, science fiction, like, as"  # Added 'like' and 'as' to avoid analogies
    )
    refined_response = get_response(refined_prompt)
    print("\nRefined response:\n", refined_response)
    
    # Evaluate the refined output
    refined_evaluation = evaluate_output(refined_response, constraints)
    print("\nRefined evaluation results:", refined_evaluation)